In [28]:
import os
import pandas as pd





# 엑셀 파일이 있는 폴더 경로
folder_path = r"C:\VISSIM_Workspace\network_test\network01_output"

# 폴더 내 모든 .xlsx 파일 찾기
xlsx_files = [f for f in os.listdir(folder_path) if f.endswith(".xlsx")]

# 모든 엑셀 파일 처리
for file in xlsx_files:
    file_path = os.path.join(folder_path, file)

    # 엑셀 파일 읽기
    df = pd.read_excel(file_path)

    # "SIMRUN" 컬럼 값 변경
    if "SIMRUN" in df.columns:
        df["SIMRUN"] = 10  # 모든 값을 10으로 변경

        # 변경된 데이터 다시 저장 (덮어쓰기)
        df.to_excel(file_path, index=False)
        print(f"Updated: {file}")
    else:
        print(f"Skipping {file} (No 'SIMRUN' column)")

print("✅ 모든 파일 처리가 완료되었습니다.")


Updated: 10_output.xlsx
Updated: 11_output.xlsx
Updated: 12_output.xlsx
Updated: 13_output.xlsx
Updated: 14_output.xlsx
Updated: 15_output.xlsx
Updated: 16_output.xlsx
Updated: 17_output.xlsx
Updated: 18_output.xlsx
Updated: 19_output.xlsx
Updated: 1_output.xlsx
Updated: 20_output.xlsx
Updated: 21_output.xlsx
Updated: 22_output.xlsx
Updated: 23_output.xlsx
Updated: 24_output.xlsx
Updated: 25_output.xlsx
Updated: 26_output.xlsx
Updated: 27_output.xlsx
Updated: 28_output.xlsx
Updated: 29_output.xlsx
Updated: 2_output.xlsx
Updated: 30_output.xlsx
Updated: 31_output.xlsx
Updated: 32_output.xlsx
Updated: 33_output.xlsx
Updated: 34_output.xlsx
Updated: 35_output.xlsx
Updated: 36_output.xlsx
Updated: 37_output.xlsx
Updated: 38_output.xlsx
Updated: 39_output.xlsx
Updated: 3_output.xlsx
Updated: 40_output.xlsx
Updated: 41_output.xlsx
Updated: 42_output.xlsx
Updated: 43_output.xlsx
Updated: 44_output.xlsx
Updated: 45_output.xlsx
Updated: 46_output.xlsx
Updated: 47_output.xlsx
Updated: 48_output.

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\VISSIM_Workspace\\network_test\\network01_output\\~$1_output.xlsx'